<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Benchmark with Movielens dataset

This notebooks compares several algorithms on Movielens dataset.



## 0 Globals settings

In [24]:
import sys
sys.path.append("../")
import os
import pandas as pd
import numpy as np
import seaborn as sns
import pyspark
import torch
import fastai
import tensorflow as tf
import surprise

from reco_utils.common.general_utils import get_number_processors
from reco_utils.common.gpu_utils import get_cuda_version, get_cudnn_version
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_chrono_split

from benchmark_utils import * #TODO: change this

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("PySpark version: {}".format(pyspark.__version__))
print("Surprise version: {}".format(surprise.__version__))
print("PyTorch version: {}".format(torch.__version__))
print("Fast AI version: {}".format(fastai.__version__))
print("Tensorflow version: {}".format(tf.__version__))
print("CUDA version: {}".format(get_cuda_version()))
print("CuDNN version: {}".format(get_cudnn_version()))
n_cores = get_number_processors()
print("Number of cores: {}".format(n_cores))

%load_ext autoreload
%autoreload 2

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Pandas version: 0.24.1
PySpark version: 2.3.1
Surprise version: 1.0.6
PyTorch version: 1.0.0
Fast AI version: 1.0.42
Tensorflow version: 1.12.0
CUDA version: CUDA Version 9.2.148
CuDNN version: 7.2.1
Number of cores: 6
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%env PYSPARK_PYTHON=/anaconda/envs/reco_full/bin/python
%env PYSPARK_DRIVER_PYTHON=/anaconda/envs/reco_full/bin/python

In [3]:
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS_CPU = 30
EPOCHS_PYSPARK = 15
EPOCHS_GPU = 5


In [4]:
# Hide fastai progress bar
hide_fastai_progress_bar()

In [5]:
# fix random seeds to make sure out runs are reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [6]:
environments = {
    "als": "pyspark",
    "sar": "python_cpu",
    "svd": "python_cpu",
    "fastai": "python_gpu",
    "ncf": "python_gpu",
}

metrics = {
    "als": ["rating", "ranking"],
    "sar": ["ranking"],
    "svd": ["rating", "ranking"],
    "fastai": ["rating", "ranking"],
    "ncf": ["ranking"],
}

In [7]:
als_params = {
    "rank": 10,
    "maxIter": EPOCHS_PYSPARK,
    "implicitPrefs": False,
    "alpha": 0.1,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False,
    "userCol": USER_COL,
    "itemCol": ITEM_COL,
    "ratingCol": RATING_COL,
}

sar_params = {
    "remove_seen": True,
    "similarity_type": "jaccard",
    "time_decay_coefficient": 30,
    "time_now": None,
    "timedecay_formula": True,
    "col_user": USER_COL,
    "col_item": ITEM_COL,
    "col_rating": RATING_COL,
    "col_timestamp": TIMESTAMP_COL,
}

svd_params = {
    "n_factors": 200,
    "n_epochs": EPOCHS_CPU,
    "lr_all": 0.005,
    "reg_all": 0.02,
    "random_state": SEED,
    "verbose": False
}

fastai_params = {
    "n_factors": 40, 
    "y_range": [0,5.5], 
    "wd": 1e-1,
    "max_lr": 5e-3,
    "epochs": EPOCHS_GPU
}

ncf_params = {
    "model_type": "NeuMF",
    "n_factors": 4,
    "layer_sizes": [16,8,4],
    "n_epochs": EPOCHS_GPU,
    "batch_size": 1024,
    "learning_rate": 1e-3,
    "verbose": 10
}


params = {
    "als": als_params,
    "sar": sar_params,
    "svd": svd_params,
    "fastai": fastai_params,
    "ncf": ncf_params,
}

In [8]:
prepare_training_data = {
    "als": prepare_training_als,
    "svd": prepare_training_svd,
    "fastai": prepare_training_fastai,
    "ncf": prepare_training_ncf,
}

In [9]:
prepare_metrics_data = {
    "als": lambda train, test: prepare_metrics_als(train, test),
    "fastai": lambda train, test: prepare_metrics_fastai(train, test),    
}

In [10]:
trainer = {
    "als": lambda params, data: train_als(params, data),
    "svd": lambda params, data: train_svd(params, data),
    "sar": lambda params, data: train_sar(params, data), 
    "fastai": lambda params, data: train_fastai(params, data),
    "ncf": lambda params, data: train_ncf(params, data),
}

In [11]:
rating_predictor = {
    "als": lambda model, test: predict_als(model, test),
    "svd": lambda model, test: predict_svd(model, test),
    "fastai": lambda model, test: predict_fastai(model, test),
}

In [12]:
ranking_predictor = {
    "als": lambda model, test, train: recommend_k_als(model, test, train),
    "sar": lambda model, test, train: recommend_k_sar(model, test, train),
    "svd": lambda model, test, train: recommend_k_svd(model, test, train),
    "fastai": lambda model, test, train: recommend_k_fastai(model, test, train),
    "ncf": lambda model, test, train: recommend_k_ncf(model, test, train),
}

In [13]:
rating_evaluator = {
    "als": lambda test, predictions: rating_metrics_pyspark(test, predictions),
    "svd": lambda test, predictions: rating_metrics_python(test, predictions),
    "fastai": lambda test, predictions: rating_metrics_python(test, predictions)
}
    
    
ranking_evaluator = {
    "als": lambda test, predictions, k: ranking_metrics_pyspark(test, predictions, k),
    "sar": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "svd": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "fastai": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "ncf": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}

In [14]:
def generate_summary(data, algo, k, train_time, time_rating=np.nan, rating_metrics=None, time_ranking=np.nan, ranking_metrics=None):
    summary = {"Data": data, "Algo": algo, "K": k, "Train time": train_time, "Rating time": time_rating, "Ranking time": time_ranking}
    if rating_metrics is None:
        rating_metrics = {
            "RMSE": np.nan,
            "MAE": np.nan,
            "R2": np.nan,
            "Explained Variance": np.nan,
        }
    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(rating_metrics)
    summary.update(ranking_metrics)
    return summary

In [18]:
data_sizes = ["100k"]#, "1m"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = ["fastai"]#["als", "svd", "sar", "ncf", "fastai"]

In [ ]:
%%time

# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time", "Rating time", "RMSE", "MAE", "R2", "Explained Variance", "Ranking time", "MAP", "nDCG@k", "Precision@k", "Recall@k", ]
df_results = pd.DataFrame(columns=cols)

for data_size in data_sizes:
    # Load the dataset
    df = movielens.load_pandas_df(
        size=data_size,
        header=[USER_COL, ITEM_COL, RATING_COL, TIMESTAMP_COL]
    )
    print("Size of Movielens {}: {}".format(data_size, df.shape))
    
    # Split the dataset
    df_train, df_test = python_chrono_split(df, 
                                  ratio=0.75, 
                                  min_rating=1, 
                                  filter_by="user", 
                                  col_user=USER_COL, 
                                  col_item=ITEM_COL, 
                                  col_timestamp=TIMESTAMP_COL)
    print("Train set size: {}".format(df_train.shape))
    print("Test set size: {}".format(df_test.shape))
   
    # Loop through the algos
    for algo in algorithms:
        print("\nComputing {} algorithm on Movielens {}".format(algo, data_size))
          
        # Data prep for training set
        train = prepare_training_data.get(algo, lambda x:x)(df_train)
        
        # Get model parameters
        model_params = params[algo]
          
        # Train the model
        model, time_train = trainer[algo](model_params, train)
        print("Training time: {}".format(time_train))
                
        # Predict and evaluate
        print("\nEvaluating with {}".format(algo))
        train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
        
        if "rating" in metrics[algo]:   
            # Predict for rating
            preds, time_rating = rating_predictor[algo](model, test)
            print("Rating prediction time: {}".format(time_rating))
                        
            # Evaluate for rating
            ratings = rating_evaluator[algo](test, preds)
            print("Rating metrics: \n{}".format(json.dumps(ratings, indent=4, sort_keys=True)))
        else:
            ratings = None
            time_rating = None
        
        if "ranking" in metrics[algo]:
            # Predict for ranking
            top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
            print("Ranking prediction time: {}".format(time_ranking))
            
            # Evaluate for rating
            rankings = ranking_evaluator[algo](test, top_k_scores, TOP_K)
            print("Ranking metrics: \n{}".format(json.dumps(rankings, indent=4, sort_keys=True)))
        else:
            rankings = None
            time_ranking = None
            
        # Record results
        summary = generate_summary(data_size, algo, TOP_K, time_train, time_rating, ratings, time_ranking, rankings)
        df_results.loc[df_results.shape[0] + 1] = summary

4.93MB [00:01, 4.41MB/s]                            


Size of Movielens 100k: (100000, 4)
Train set size: (74992, 4)
Test set size: (25008, 4)

Computing fastai algorithm on Movielens 100k


In [17]:
df_results

,Data,Algo,K,Train time,Rating time,RMSE,MAE,R2,Explained Variance,Ranking time,MAP,nDCG@k,Precision@k,Recall@k
1,100k,als,10,0:01:05.192551,0:00:00.039407,1.026328,0.801722,0.241869,0.234578,0:00:00.068504,0.002479,0.023425,0.026087,0.009980
2,100k,svd,10,0:00:09.224441,0:00:02.569490,1.000157,0.791798,0.275695,0.280291,0:00:12.386970,0.010260,0.066190,0.060870,0.025866
3,100k,sar_single_node,10,0:00:00.189971,None,NaN,NaN,NaN,NaN,0:00:00.097058,0.004793,0.029888,0.034677,0.024225
4,100k,ncf,10,0:00:20.162137,None,NaN,NaN,NaN,NaN,0:00:02.586475,0.033134,0.162691,0.143054,0.064987
